In [1]:
import itertools
import operator
import re
from abc import *
from copy import deepcopy
from operator import itemgetter
from typing import *
from nodestimation.learning.connectoming import *
from nodestimation.learning.modification import normalize_df
import numpy as np
import scipy as sp
import networkx as nx
import mne
import time
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors._dist_metrics import DistanceMetric
from sklearn.utils import shuffle
from typing import List, Tuple
from sklearn.model_selection import train_test_split
from nodestimation.learning.estimation import collect_statistic, \
    compute_importance, collect_cross_statistic, make_selection_map, \
    select, separate_datasets, selected_statistic, choose_best, selected_data, make_feature_selection
from nodestimation.learning.informativeness import CrossInformativeness, Informativeness, SubjectsInformativeness, \
    NodesInformativeness
from nodestimation.learning.networking import sparse_graph, graph_to_hemispheres, hemispheres_division_modularity, \
    metric_for_hemispheres
from nodestimation.processing.features import prepare_features
from nodestimation.project import find_subject_dir, conditions_unique_code
from nodestimation.pipeline import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import nibabel
import nilearn.plotting as nplt
from nodestimation.project.actions import read
import nodestimation as nd
from nodestimation.learning.modification import append_series, promote
import nodestimation.learning.modification as lmd
from nodestimation.project.subject import Subject
from sklearn.preprocessing import *
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectFromModel
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
import seaborn as sns
import matplotlib as mpl
from nodestimation.learning.selection import SubjectsStatistic, Wilcoxon, Mannwhitneyu, Test
from scipy.stats import wilcoxon
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from scipy.fftpack import fftfreq, irfft, rfft
from scipy.fftpack import fftfreq, irfft, rfft

/home/user/PycharmProjects/NodesEstimation/venv/lib/python3.9/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
SUBJECTS = pipeline(
    methods=['wpli', 'envelope', 'coh', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli', 'pli2_unbiased', 'wpli2_debiased'],
    freq_bands=(4, 8),
    centrality_metrics=['eigen', 'between', 'degree', 'info']
    )

All computation has been already done, loading of the existing file with the solution...


In [3]:
#tests section

CONNECTOMES_KINDS = [
    'initial',
    ## 'binary',
    ## 'suppressed',
    # 'inverse',
    # 'inverse_binary',
    # 'inverse_suppressed',
    ## 'spatial',
    # 'initial&spatial',
    # 'inverse&spatial',
    ## 'bin&spatial',
    ## 'supp&spatial',
    # 'inverse-bin&spatial',
    # 'inverse-supp&spatial'

]

for CONNECTOMES_KIND in  CONNECTOMES_KINDS:

    print(f'-------------------------------------------------------\n'
          f'{CONNECTOMES_KIND.capitalize()} connectomes computation\n'
          f'-------------------------------------------------------\n')

    subjects = SUBJECTS.copy()

    datasets = {
        subject.name: {
            freq: {
                method: make_connectome(subject, freq, method, CONNECTOMES_KIND, threshold=1)
               for method in subject.connectomes[freq]
            }
            for freq in subject.connectomes
        }
        for subject in subjects
    }
    print('Connectomes are prepared')

    close, between, eigen, degree, info, harmony = dict(), dict(), dict(), dict(), dict(), dict()

    total_ignored_subjects = 0

    for subject_name in datasets:
        is_corrupted = False
        close.update({subject_name: dict()})
        between.update({subject_name: dict()})
        eigen.update({subject_name: dict()})
        degree.update({subject_name: dict()})
        info.update({subject_name: dict()})
        harmony.update({subject_name: dict()})
        # katz.update({subject_name: dict()})
        for freq in datasets[subject_name]:
            if is_corrupted:
                break
            for method in datasets[subject_name][freq]:
                print(subject_name, freq, method)
                label_names = datasets[subject_name][freq][method].columns

                # G = nx.convert_matrix.from_numpy_array(
                #             datasets[subject_name][freq][method].to_numpy()
                #         )
                # nx.draw(G)
                # plt.show()

                if nx.is_connected(
                        nx.convert_matrix.from_numpy_array(
                            datasets[subject_name][freq][method].to_numpy()
                        )
                ):
                    arr  = datasets[subject_name][freq][method].to_numpy()
                else:
                    corrupted_subject = [subject for subject in subjects if subject.name == subject_name][0]
                    subjects.remove(corrupted_subject)
                    total_ignored_subjects += 1
                    print(f'Graph is not connected: {subject_name}, {freq}, {method}.\n'
                          f'This subject will be ignored.\n'
                          f'Total of ignored subjects: {total_ignored_subjects}\n'
                          f'Total of spared subjects: {len(subjects)}\n')
                    is_corrupted = True
                    break
                    # raise ValueError(f'Graph is not connected: {subject_name}, {freq}, {method}')
                # if matrix has negative values, shift it to make minimal value at least 0
                if arr.min().min() < 0:
                    print(f'\tShifting connectome for {method} at {freq}')
                    for i in range(arr.shape[0]):
                        for j in range(arr.shape[1]):
                            if i != j: arr[i, j] -= arr.min().min()
                G = nx.convert_matrix.from_numpy_array(arr)
                mapping = {node: label_name for node, label_name in zip(G, label_names)}
                G = nx.relabel_nodes(G, mapping)
                if CONNECTOMES_KIND == 'initial' or CONNECTOMES_KIND == 'inverse' or CONNECTOMES_KIND == 'spatial':
                    G_sup = nx.convert_matrix.from_numpy_array(
                            lmd.suppress(
                                datasets[subject_name][freq][method],
                                trigger= arr.mean().mean(),
                                optimal=0
                            ).to_numpy()
                        )
                    G_sup = nx.relabel_nodes(G_sup, mapping)
                else:
                    G_sup = G
                for place, data in zip(
                    [
                        close[subject_name],
                        between[subject_name],
                        eigen[subject_name],
                        info[subject_name],
                        harmony[subject_name],
                        degree[subject_name],
                        # katz[subject]
                    ],[
                            nx.closeness_centrality(G, distance='weight'),
                            nx.betweenness_centrality(G_sup, weight='weight'),
                            nx.eigenvector_centrality_numpy(G, weight='weight'),
                            nx.information_centrality(G, weight='weight'),
                            nx.harmonic_centrality(G, distance='weight'),
                            dict(G.degree(weight='weight')),
                            # nx.katz_centrality(G, weight='weight', max_iter=100)
                        ]
                ):
                    place.update({
                        freq + '_' + method: pd.Series(data)
                    })

    print('All is done')

    datasets_centrality = dict()

    for subject in subjects:
        datasets_centrality.update({subject.name: dict()})
        datasets_centrality[subject.name].update({
            'close': pd.DataFrame(close[subject.name]),
            'between': pd.DataFrame(between[subject.name]),
            'eigen': pd.DataFrame(eigen[subject.name]),
            'degree': pd.DataFrame(degree[subject.name]),
            'info': pd.DataFrame(info[subject.name]),
            'harmony': pd.DataFrame(harmony[subject.name]),
            # 'katz': pd.DataFrame(katz[subject.name])
        })
        true = subject.datasets['eigen']['resected']
        datasets_centrality[subject.name]['close'] = datasets_centrality[subject.name]['close'].assign(resected=true)
        datasets_centrality[subject.name]['between'] = datasets_centrality[subject.name]['between'].assign(resected=true)
        datasets_centrality[subject.name]['eigen'] = datasets_centrality[subject.name]['eigen'].assign(resected=true)
        datasets_centrality[subject.name]['degree'] = datasets_centrality[subject.name]['degree'].assign(resected=true)
        datasets_centrality[subject.name]['info'] = datasets_centrality[subject.name]['info'].assign(resected=true)
        datasets_centrality[subject.name]['harmony'] = datasets_centrality[subject.name]['harmony'].assign(resected=true)
        # datasets_centrality[subject.name]['katz'] = datasets_centrality[subject.name]['katz'].assign(resected=true)

    for subject in subjects:
        subject.datasets = datasets_centrality[subject.name]

    print(f'Connectomes successfully updated. Now each subject has new {CONNECTOMES_KIND} connectomes')

    stat1 = SubjectsStatistic(subjects, 'resected', centrality_metric='eigen')
    print('Eigencentrality statistics are collected')
    stat2 = SubjectsStatistic(subjects, 'resected', centrality_metric='between')
    print('Betweenness centrality statistics are collected')
    stat3 = SubjectsStatistic(subjects, 'resected', centrality_metric='close')
    print('Closeness centrality statistics are collected')
    stat4 = SubjectsStatistic(subjects, 'resected', centrality_metric='degree')
    print('Degree centrality statistics are collected')
    stat5 = SubjectsStatistic(subjects, 'resected', centrality_metric='info')
    print('Info centrality statistics are collected')
    stat6 = SubjectsStatistic(subjects, 'resected', centrality_metric='harmony')
    print('Harmony centrality statistics are collected')
    # stat7 = SubjectsStatistic(subjects, 'resected', centrality_metric='katz')
    print(f'All statistics for {CONNECTOMES_KIND} connectomes are collected')

    test1 = stat1.test(state='reflected')
    test2 = stat2.test(state='reflected')
    test3 = stat3.test(state='reflected')
    test4 = stat4.test(state='reflected')
    test5 = stat5.test(state='reflected')
    test6 = stat6.test(state='reflected')
    test1_samples, test2_samples, test3_samples, test4_samples, test5_samples, test6_samples = list(), list(), list(), list(), list(), list()
    for feature in test1.result:
        test1_samples.append(test1.result[feature][1])
        test2_samples.append(test2.result[feature][1])
        test3_samples.append(test3.result[feature][1])
        test4_samples.append(test4.result[feature][1])
        test5_samples.append(test5.result[feature][1])
        test6_samples.append(test6.result[feature][1])

    test_samples = np.array([
        np.array(test1_samples),
        np.array(test2_samples),
        np.array(test3_samples),
        np.array(test4_samples),
        np.array(test5_samples),
        np.array(test6_samples)
    ])

    df = pd.DataFrame(test_samples, columns=list(test1.result.keys()), index=[
        'eigen', 'between', 'close', 'degree', 'info', 'harmony'
    ]).T
    print(df)
    df.to_csv(f'/home/user/Documents/Wilcoxon_tests_for_{CONNECTOMES_KIND}_connectomes.csv')

    test1 = stat1.test(state='reflected', test='mannwhitneyu')
    test2 = stat2.test(state='reflected', test='mannwhitneyu')
    test3 = stat3.test(state='reflected', test='mannwhitneyu')
    test4 = stat4.test(state='reflected', test='mannwhitneyu')
    test5 = stat5.test(state='reflected', test='mannwhitneyu')
    test6 = stat6.test(state='reflected', test='mannwhitneyu')
    test1_samples, test2_samples, test3_samples, test4_samples, test5_samples, test6_samples = list(), list(), list(), list(), list(), list()
    for feature in test1.result:
        test1_samples.append(test1.result[feature][1])
        test2_samples.append(test2.result[feature][1])
        test3_samples.append(test3.result[feature][1])
        test4_samples.append(test4.result[feature][1])
        test5_samples.append(test5.result[feature][1])
        test6_samples.append(test6.result[feature][1])

    test_samples = np.array([
        np.array(test1_samples),
        np.array(test2_samples),
        np.array(test3_samples),
        np.array(test4_samples),
        np.array(test5_samples),
        np.array(test6_samples)
    ])

    df = pd.DataFrame(test_samples, columns=list(test1.result.keys()), index=[
        'eigen', 'between', 'close', 'degree', 'info', 'harmony'
    ]).T
    print(df)
    df.to_csv(f'/home/user/Documents/MannWhitneyu_tests_for_{CONNECTOMES_KIND}_connectomes.csv')

-------------------------------------------------------
Initial connectomes computation
-------------------------------------------------------

Connectomes are prepared
P1H2 4-8Hz wpli
P1H2 4-8Hz envelope
P1H2 4-8Hz coh
P1H2 4-8Hz imcoh
	Shifting connectome for imcoh at 4-8Hz
P1H2 4-8Hz plv
P1H2 4-8Hz ciplv
P1H2 4-8Hz ppc
	Shifting connectome for ppc at 4-8Hz
P1H2 4-8Hz pli
P1H2 4-8Hz pli2_unbiased
	Shifting connectome for pli2_unbiased at 4-8Hz
P1H2 4-8Hz wpli2_debiased
	Shifting connectome for wpli2_debiased at 4-8Hz
M2S2 4-8Hz wpli
M2S2 4-8Hz envelope
M2S2 4-8Hz coh
M2S2 4-8Hz imcoh
	Shifting connectome for imcoh at 4-8Hz
M2S2 4-8Hz plv
M2S2 4-8Hz ciplv
M2S2 4-8Hz ppc
	Shifting connectome for ppc at 4-8Hz
M2S2 4-8Hz pli
M2S2 4-8Hz pli2_unbiased
	Shifting connectome for pli2_unbiased at 4-8Hz
M2S2 4-8Hz wpli2_debiased
	Shifting connectome for wpli2_debiased at 4-8Hz
R1D2 4-8Hz wpli
R1D2 4-8Hz envelope
R1D2 4-8Hz coh
R1D2 4-8Hz imcoh
	Shifting connectome for imcoh at 4-8Hz
R1D2 4-8Hz

In [ ]:
connectome = Connectome(SUBJECTS[0], '4-8Hz', 'plv')
label_names = connectome.matrix.columns
# n = np.linalg.norm(connectome.matrix)
m = connectome.inverse()
G = nx.convert_matrix.from_numpy_array(
    lmd.binarize(
        m,
        trigger= m.to_numpy().mean().mean(),
    ).to_numpy()
    # connectome.inverse().to_numpy()
)
mapping = {node: label_name for node, label_name in zip(G, label_names)}
G = nx.relabel_nodes(G, mapping)
# nx.draw(G)
# plt.plot()

In [ ]:
con = make_connectome(SUBJECTS[1], '4-8Hz', 'ppc', 'inverse_binary')

G = nx.convert_matrix.from_numpy_array(con.to_numpy())
G = nx.relabel_nodes(G, mapping)
print(nx.is_connected(G))
# nx.draw(G)
# plt.plot()


p = nx.betweenness_centrality(G, weight='weight')
# print(lmd.dict_to_str(p))


In [ ]:
print('spatial map')
plt.imshow(
    connectome.spatial_map().to_numpy()
)
plt.show()

inv_connectome = connectome.inverse()
spatial_map = connectome.spatial_map()

print('connectome')
plt.imshow(
    connectome.matrix.to_numpy()
)
plt.show()

print('inv connectome')
plt.imshow(
    inv_connectome.to_numpy()
)
plt.show()

print('inv connectome * spatial map')
plt.imshow(
    spatial_map.to_numpy() *
    inv_connectome.to_numpy()
)
plt.show()

print('sup inv connectome * spatial map')
plt.imshow(
    spatial_map.to_numpy() *
    lmd.suppress(
        inv_connectome,
        trigger=inv_connectome.to_numpy().mean().mean(),
        optimal=0
    ).to_numpy()
)
plt.show()

In [ ]:
connectome.plot(title='initial')
connectome.plot(inv_connectome, title='inverse')
connectome.plot(spatial_map, title='spatial')
connectome.plot(inv_connectome*spatial_map, title='inverse&spatial')
connectome.plot(
    lmd.suppress(
        inv_connectome,
        trigger=inv_connectome.to_numpy().mean().mean(),
        optimal=0)*
    spatial_map,
    title='(suppressed&inverse)&spatial'
)
connectome.plot(
    lmd.suppress(
        inv_connectome*spatial_map,
        trigger=(inv_connectome*spatial_map).to_numpy().mean().mean(),
        optimal=0),
    title='suppressed&(inverse&spatial)'
)

In [ ]:
connectome.plot(normalize_df(inv_connectome)*normalize_df(spatial_map), title='inverse&spatial, normalized')

connectome.plot(
    normalize_df(lmd.suppress(
        inv_connectome,
        trigger=inv_connectome.to_numpy().mean().mean(),
        optimal=0))*
    normalize_df(spatial_map),
    title='(suppressed&inverse)&spatial, normalized'
)



# connectome.plot(
#     lmd.suppress(
#         normalize_df(inv_connectome)*normalize_df(spatial_map),
#         trigger=(inv_connectome*spatial_map).to_numpy().mean().mean(),
#         optimal=0),
#     title='supressed&(inverse&spatial), normalized'
# )
